In [1]:
import pandas as pd
import umap
import umap.plot
import hdbscan
import numpy as np

from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers import models, losses
from torch.utils.data import DataLoader
from sklearn.metrics import v_measure_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

In [2]:
data = pd.read_csv("C:/Users/Administrator/Desktop/all_original_text_and_headline_news.csv", encoding="utf8")
data.tail()

,Unnamed: 0,All_headline_from_every_reference,All_content_from_every_reference,All_URL_from_every_reference
6685,6685,ชัวร์ก่อนแชร์ : ระวัง 10 ขนมขบเคี้ยวอันตราย! จ...,สำนักข่าวไทย 13 ก.พ.-บนโลกออนไลน์ มีการแชร์กัน...,https://tna.mcot.net/tna-93789
6686,6686,ชัวร์ก่อนแชร์ : เปิดแอร์ใส่หน้าเด็ก ทำให้เป็นอ...,กทม. 6 ก.พ. – โลกออนไลน์เตือนให้ระวังการให้ลู...,https://tna.mcot.net/tna-93790
6687,6687,ชัวร์ก่อนแชร์ : มะเร็งต้องเลิกบำรุงจริงหรือ?,กทม. 30 ม.ค. – สังคมออนไลน์แชร์กันว่าให้คนเป็...,https://tna.mcot.net/tna-93791
6688,6688,ชัวร์ก่อนแชร์ : ยาแอสไพรินช่วยซักผ้าขาวสะอาด จ...,กทม. 12 เม.ย.-บนสังคมออนไลน์แชร์แนะนำเคล็ดลับซ...,https://tna.mcot.net/tna-100770
6689,6689,ชัวร์ก่อนแชร์ : เทคนิคซักผ้าด้วยเครื่องไม่ทำให...,กทม. 14 เม.ย.-บนสังคมออนไลน์แชร์คลิปเทคนิคการซ...,https://tna.mcot.net/tna-101176


In [3]:
reviews = data.All_headline_from_every_reference.values.tolist()

In [4]:
reviews

['น้ำชาไม่สามารถรักษาโรคแพ้อาหารได้ โปรดอย่าหลงเชื่อ',
 'มะนาวเกลือแก้ไมเกรน มั่วมาก อย่าไปเชื่อ',
 'อย่าหลงเชื่อ “นาโน สเปรย์” อ้างฆ่าเชื้อไวรัสและแบคทีเรีย',
 'ดื่มกาแฟวันละสามแก้วทำให้หน้าอกเล็กลงไม่จริง ',
 'คิ้วดกดำด้วยการทาปิโตรเลียม เจลลี่ ไม่จริง',
 'อย่าหลงเชื่อ! ‘อ.เจษฎ์’ ยัน ‘ไวรัสโคโรนา’ ถูกตัดต่อพันธุกรรมผสม HIV เป็นอาวุธชีวภาพ ไม่เป็นความจริง',
 'ข่าวปลอม อย่าแชร์! ยาพาราเซตามอลมีส่วนผสมของไวรัสแมคชูโป',
 'อย.ย้ำข่าวยาพาราปนเปื้อนไวรัส ไม่เป็นความจริง',
 'เพจดัง โต้ พืชกระท่อมต้านโควิด ทึ่ง ผลข้างเคียงเพียบ-ถอนยาทรมานสุดๆ',
 'หมอมนูญชี้โควิด-19 แพร่ทางอากาศได้ จี้ดีอีเอสแก้ไขข้อมูล',
 'ผู้เชี่ยวชาญไม่แนะนำเอาทิชชู่มาทำหน้ากากอนามัย ชี้สุดอันตราย',
 'ข่าวปลอม อย่าแชร์! กลั้นหายใจ เช็คอาการไอ-แน่นหน้าอก ตรวจการติดเชื้อ ‘ไวรัสโคโรนา’ ด้วยตัวเอง',
 'ข่าวปลอม อย่าแชร์! กลั้นหายใจ เช็คอาการไอ-แน่นหน้าอก ตรวจการติดเชื้อ ‘ไวรัสโคโรนา’ ด้วยตัวเอง',
 'อย่าเชื่อ! ดื่มน้ำอุ่น กินกระเทียม ป้องกันโรคโควิด-19\xa0',
 'เชื้อไวรัสโคโรนาสายพันธุ์ใหม่ สามารถติดต่อผ่านการมองตาได้ ข่าวปลอม สร้

In [5]:
for i in range(len(reviews)):
  if type(reviews[i]) != type("string"):
    reviews[i] = str(reviews[i])

In [6]:
# model_name = 'mrp/simcse-model-roberta-base-thai'

# word_embedding_model = models.Transformer(model_name, max_seq_length=256)
# pooling_model = models.Pooling(
#     word_embedding_model.get_word_embedding_dimension(),
#     pooling_mode_mean_tokens=True
# )
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [7]:
# # Convert train sentences to positive pairs
# train_data = [InputExample(texts=[text, text]) for text in reviews]

# # DataLoader to batch your data
# train_dataloader = DataLoader(train_data, batch_size=2, shuffle=True)

# # Use the MultipleNegativesRankingLoss
# train_loss = losses.MultipleNegativesRankingLoss(model)

In [8]:
# # Call the fit method
# model.fit(
#     train_objectives=[(train_dataloader, train_loss)],
#     epochs=2,
#     show_progress_bar=True
# )

# model.save('output/simcse-model')

In [9]:
# import torch

# del temp
# torch.cuda.empty_cache()

In [10]:
# load the trained SimCSE model
model = SentenceTransformer("./output/simcse-model/")

In [11]:
# encode the reviews
cse_review_encodings = model.encode(reviews)

In [12]:
cse_review_encodings

array([[ 0.4256501 , -0.05324041,  0.23133872, ...,  1.4037349 ,
         0.15373808,  1.165051  ],
       [ 0.5231712 , -0.50408137,  0.05902619, ...,  0.36494097,
         0.37483454,  0.8073988 ],
       [-0.21491972,  0.0189983 , -0.07054427, ...,  0.08570265,
        -0.4948245 , -0.7696825 ],
       ...,
       [-0.04427978,  0.48717403,  0.3253145 , ..., -0.18065572,
         0.40573177, -0.36198398],
       [ 0.3620997 , -0.4371709 ,  0.0220499 , ...,  0.84054846,
        -0.7720446 , -0.41769424],
       [-0.20563582, -0.5493508 ,  0.49121886, ..., -0.04631548,
        -0.26303458, -0.8310114 ]], dtype=float32)

In [13]:
# mapper = umap.UMAP().fit(cse_review_encodings)

# p = umap.plot.points(mapper)
# umap.plot.show(p)

In [72]:
all_result_data = pd.read_csv("C:/Users/Administrator/Desktop/all_result_with_url.csv")
all_result_data.tail()

,Unnamed: 0,index,headline,url,content
26,26,4199,ข่าวปลอม อย่าแชร์! คลิปจากต่างประเทศแนะดื่มน้ำ...,https://www.antifakenewscenter.com/news/activi...,ตามที่ได้มีคลิปวิดีโอปรากฎในสื่อออนไลน์ต่างๆ ใ...
27,27,1416,ข่าวปลอม อย่าแชร์! คลิปจากต่างประเทศแนะดื่มน้ำ...,https://www.antifakenewscenter.com/%E0%B8%82%E...,ตามที่ได้มีคลิปวิดีโอปรากฎในสื่อออนไลน์ต่างๆ ใ...
28,28,3452,ข่าวปลอม อย่าแชร์! คลิปจากต่างประเทศแนะดื่มน้ำ...,https://www.antifakenewscenter.com/news/activi...,ตามที่ได้มีคลิปวิดีโอปรากฎในสื่อออนไลน์ต่างๆ ใ...
29,29,1540,ข่าวปลอม อย่าแชร์! คลิปจากต่างประเทศแนะดื่มน้ำ...,https://www.antifakenewscenter.com/%E0%B8%82%E...,ตามที่ได้มีคลิปวิดีโอปรากฎในสื่อออนไลน์ต่างๆ ใ...
30,30,-1,กรุงเทพธุรกิจ covld-19จีน' เปิดตัววัคซีนโควิด-...,NaN,NaN


In [73]:
all_result_data.head()

,Unnamed: 0,index,headline,url,content
0,0,2250,จีนเปิดตัววัคซีนต้านโควิด-19 แบบสูดเข้าปอดครั้...,https://www.tnnthailand.com/news/covid19/96445/,วันนี้ ( 13 พ.ย. 64 )วัคซีนป้องกันโควิด-19 ชนิ...
1,1,494,ครั้งแรกของโลก! ปลูกถ่ายสเต็มเซลล์เร่งด่วนจากผ...,https://news.thaipbs.or.th/content/293953,หมอรามาธิบดี ประสบความความสำเร็จ ปลูกถ่ายสเต็ม...
2,2,660,"ญี่ปุ่นพบสัตว์เลี้ยงติดเชื้อ ""โควิด"" ครั้งแรกข...",https://www.thansettakij.com/content/world/444...,ซึ่งเป็นบริษัทประกันชีวิตสัตว์เลี้ยงในญี่ปุ่น ...
3,3,548,"อินเดียเล็งเปิดตัว ""วัคซีน COVID-19"" ตัวแรก 15...",https://news.thaipbs.or.th/content/294277,สภาวิจัยทางการแพทย์แห่งอินเดีย (ICMR) เตรียมเป...
4,4,3888,ข่าวปลอม อย่าแชร์! รพ.กรุงเทพ เปิดลงทะเบียนจอง...,https://www.antifakenewscenter.com/news/activi...,ตามที่มีการกล่าวถึงในประเด็นเรื่อง รพ.กรุงเทพ ...


In [74]:
result_headline = all_result_data.headline.values.tolist()

In [75]:
result_headline_encodings = model.encode(result_headline)

In [76]:
result_headline_encodings

array([[-0.71265274,  0.46743906, -0.06933784, ...,  0.56222034,
        -0.7236973 ,  1.1720409 ],
       [-0.2838697 ,  0.38820145, -0.24625997, ..., -0.83666813,
        -0.12268759, -0.81569463],
       [ 0.09590595,  0.4625698 , -0.04467639, ..., -0.6247978 ,
        -0.18828532,  0.10652603],
       ...,
       [-0.02754137,  0.2577714 , -0.5164914 , ...,  0.27423626,
        -0.50373083,  0.83410573],
       [-0.02754137,  0.2577714 , -0.5164914 , ...,  0.27423626,
        -0.50373083,  0.83410573],
       [-0.04909753,  0.02331697,  0.07488269, ...,  0.9228767 ,
        -0.8701557 ,  0.6353302 ]], dtype=float32)

In [77]:
# norm_result_headline_encodings = normalize(result_headline_encodings)

In [78]:
# d_cosines = np.dot(norm_result_headline_encodings,norm_result_headline_encodings[-1])

In [79]:
# d_cosines

In [80]:
# result = []
# for i in range(len(d_cosines)):
#     tmp = []
#     tmp.append(i)
#     tmp.append(d_cosines[i])
#     result.append(tmp)
    
# print(result)

In [81]:
# # take second element for sort
# def takeSecond(elem):
#     return elem[1]

# # random list
# # random = [(2, 2), (3, 4), (4, 1), (1, 3)]

# # sort list with key
# result.sort(key=takeSecond, reverse=True)

# # print list
# print('Sorted list:', result)

In [82]:
# index_list = []
# for matched_news_index in range(len(result)):
#     index_list.append(result[matched_news_index][0])

# headline_df = pd.DataFrame
# headline_list = []
# for index in index_list:
#     headline_df = all_result_data.loc[[index], ['headline']]
#     headline_list.append(headline_df.headline.values.tolist())

In [83]:
# headline_list

In [84]:
# umap_embs = mapper.transform(result_headline_encodings)

# clusterer = hdbscan.HDBSCAN(min_samples=2, min_cluster_size=10, prediction_data=True).fit(umap_embs)
# soft_clusters = hdbscan.all_points_membership_vectors(clusterer)
# cluster_labels = [np.argmax(x) for x in soft_clusters]

In [85]:
# print(cluster_labels)

In [86]:
# cluster_labels.count(cluster_labels[-1])

In [87]:
# for i in range(len(cluster_labels)):
#   if cluster_labels[i] == cluster_labels[-1]:
#     print(i)

----------------- Try DBSCAN FROM SKLEARN ----------------------

In [88]:
from sklearn.cluster import DBSCAN
import numpy as np

X = result_headline_encodings
clustering = DBSCAN(eps=3, min_samples=2).fit(X)
clustering.labels_

array([-1, -1, -1, -1, -1, -1, -1,  0,  0, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1, -1],
      dtype=int64)

In [89]:
len(clustering.labels_)

31

In [90]:
matched_news_list = []
for index in range(len(clustering.labels_)):
    if clustering.labels_[index] == clustering.labels_[-1]:
        matched_news_list.append(index)

In [91]:
matched_news_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 30]

In [92]:
headline_df = pd.DataFrame
headline_list = []
for index in matched_news_list:
    headline_df = all_result_data.loc[[index], ['headline']]
    headline_list.append(headline_df.headline.values.tolist())

In [93]:
len(headline_list)

25

In [94]:
headline_list

[['จีนเปิดตัววัคซีนต้านโควิด-19 แบบสูดเข้าปอดครั้งแรกของโลก'],
 ['ครั้งแรกของโลก! ปลูกถ่ายสเต็มเซลล์เร่งด่วนจากผู้ป่วย COVID-19'],
 ['ญี่ปุ่นพบสัตว์เลี้ยงติดเชื้อ "โควิด" ครั้งแรกของประเทศ'],
 ['อินเดียเล็งเปิดตัว "วัคซีน COVID-19" ตัวแรก 15 ส.ค. นี้'],
 ['ข่าวปลอม อย่าแชร์! รพ.กรุงเทพ เปิดลงทะเบียนจองวัคซีนโควิด-19 และเรียกเก็บชำระเงินล่วงหน้า'],
 ['"นิวซีแลนด์" กลับมาพบผู้ติดเชื้อโควิดครั้งแรกในรอบ 24 วัน'],
 ['เชื้อโควิดลงปอด อาการเป็นอย่างไร?'],
 ['ข่าวปลอม อย่าแชร์! โรงพยาบาลกรุงเทพเปิดให้ลงทะเบียนจองวัคซีน Pfizer ผ่านเว็บ siampfizer'],
 ['ชัวร์ก่อนแชร์ : ไปสูดโอโซนเพื่อสุขภาพกันเถอะ…จริงหรือ ?'],
 ['"อังกฤษ" อนุมัติวัคซีน COVID-19 ชาติแรกของโลก'],
 ['ข่าวปลอม อย่าแชร์! ล่าสุดกรุงเทพติดเชื้อไวรัสโคโรนา 5 ราย'],
 ['ข่าวปลอม อย่าแชร์! ประเทศไทยมีผู้ติดเชื้อโควิด-19 เป็นอันดับ 1 ของโลก และมีผู้เสียชีวิตเป็นอันดับ 2 ของโลก'],
 ['วัคซีนโควิด-19 จำนวน 2 แสนโดสจากจีน ส่งถึงไทยภายในเดือน ก.พ. 64 จริงหรือ?'],
 ['ชัวร์ก่อนแชร์ : 2559 ข่าวปลอม ป่วนโลก?'],
 ['วัคซีนไข้หวัดใหญ่ สามารถฉีดวันเดี